# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 12: Recommendation System with ALS** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Equipo**: JJAE

In [8]:
import findspark
findspark.init()

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ALS-Recommendation-JJAE") \
    .master("spark://51f45cfec3c3:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

### Load the movie ratings dataset

In [10]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType

ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", LongType(), True)
])

ratings_df = spark.read.csv(
    "/home/jovyan/notebooks/data/sample_movielens_ratings.txt",
    sep="::",
    schema=ratings_schema
)

ratings_df.show(5)
ratings_df.printSchema()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     0|      2|   3.0|1424380312|
|     0|      3|   1.0|1424380312|
|     0|      5|   2.0|1424380312|
|     0|      9|   4.0|1424380312|
|     0|     11|   1.0|1424380312|
+------+-------+------+----------+
only showing top 5 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: long (nullable = true)



### Train ALS Recommendation Model

In [11]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    maxIter=10,
    regParam=0.1,
    rank=5,
    coldStartStrategy="drop"
)

model = als.fit(ratings_df)

ModuleNotFoundError: No module named 'numpy'

### Generate Recommendations

In [ ]:
user_recommendations = model.recommendForAllUsers(numItems=5)
user_recommendations.show(truncate=False)

### Evaluate Model with RMSE

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

predictions = model.transform(ratings_df)

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE): {rmse:.4f}")

In [ ]:
sc.stop()